In [32]:
# pre-main imports: 
import itertools
import logging
import pipeop
from logdecorator import log_on_start, log_on_end, log_on_error, log_exception
import wrapt
from fabric import Connection
from tqdm.notebook import tqdm #from tqdm import tqdm if not in notebook

In [33]:
# main-imports: custom decorators and functions
import shared_variables
shared_variables.enabled_wrapt_decorators = True # apply or not custom core decorators
from core_decorators import *
from core_functions import *

### project specific code

In [34]:
subset_generator

<function core_functions.subset_generator(a_generator)>

### git notebook

In [35]:
# upload notebook to gitlab/github:

to_save = '*.ipynb *.py'
#current_dir=os.getcwd()
git_save_dir=os.getcwd()

c = Connection('mercury@hgi-farm5')
with c.cd(git_save_dir):
    # github repo requires using the correct deploy key:
    c.run('git config remote.origin.url git@gn5.githostname_github_bsub_jupyter_lab:wtsi-hgi/bsub_jupyter_lab.git')
    c.run('git config --local user.name gn5')
    c.run('git config --local user.email gn5@sanger.ac.uk')
    # now can pull:
    c.run('git pull')
    # c.run('cp ' + current_dir+'/'+nb_name + ' ' + git_save_dir+'/')
    c.run('ls -ltra')

Already up-to-date.
total 36
-rw-r--r-- 1 mercury hgi   32 Jul 16 14:24 shared_variables.py
-rw-r--r-- 1 mercury hgi 1124 Jul 16 14:24 core_functions.py
-rw-r--r-- 1 mercury hgi  457 Jul 16 14:24 core_decorators.py
drwxr-sr-x 9 mercury hgi 4096 Jul 21 11:34 ..
drwxr-sr-x 2 mercury hgi 4096 Jul 21 11:34 .ipynb_checkpoints
drwxr-sr-x 2 mercury hgi 4096 Jul 21 11:35 __pycache__
-rw-r--r-- 1 mercury hgi 5704 Jul 21 12:29 template_project.ipynb
drwxr-sr-x 4 mercury hgi 4096 Jul 21 12:29 .


In [24]:
with c.cd(git_save_dir):
    # now can add, commit and push files:
    c.run('git add ' + to_save)
    c.run('git commit -m auto || true')
    c.run('git push -u origin')

[master b6c2002] auto
 1 file changed, 57 insertions(+), 50 deletions(-)


#### how ssh key works:

the `gn5.githostname_github_bsub_jupyter_lab` hostname serves my ssh key because:
```
cat /nfs/users/nfs_m/mercury/.ssh/config
Host gn5.githostname_github_bsub_jupyter_lab # used by gn5 as deploy key for github repo wtsi-hgi/bsub_jupyter_lab
    AddKeysToAgent yes                                                                              
    Hostname github.com                                                                             
    User git                                                                                        
    IdentityFile ~/.ssh/id_rsa_gn5_github_bsub_jupyter_lab            
...    
```